## KPI definition

You have now been asked to discover what key performance indicators (KPIs) will determine the success of the new design? Use at least completion rate, time spent on each step and error rates. Add any KPIs you might find relevant.

- Completion Rate: The proportion of users who reach the final 'confirm' step.
- Time Spent on Each Step: The average duration users spend on each step.
- Error Rates: If there's a step where users go back to a previous step, it may indicate confusion or an error. You should consider moving from a later step to an earlier one as an error.

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
df = pd.read_csv('/Users/josebabermejo/Ironhack/5. Quests/project_vanguard_joseba/experiment_footprints_clients.csv')
df.head()

,client_id,Variation,visitor_id,visit_id,process_step,date_time
0,9988021,Test,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,Test,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,Test,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,Test,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,Test,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04


In [17]:
# Check for duplicates
df.duplicated().sum()

4074

In [19]:
# Remove duplicated
df = df.drop_duplicates()

In [21]:
# Column-wise null values
df.isnull().sum()

client_id       0
Variation       0
visitor_id      0
visit_id        0
process_step    0
date_time       0
dtype: int64

In [11]:
df.to_csv('exp_foot_cl_tableau.csv', index=False)

# Completion Rate

In [138]:
# Creation of a new column called confirm step, that return 0 and 1, depending on the 'confirm' step
df['confirm_step'] = (df['process_step'] == 'confirm').astype(int)
df

,client_id,Variation,visitor_id,visit_id,process_step,date_time,time_diff,time_diff_min,step_num,prev_step_num,prev_step_name,error_flag,error_transition,confirm_step
142182,3561384,Test,451664975_1722933822,100012776_37918976071_457913,confirm,2017-04-26 13:22:17,0 days 00:00:00,0.0,4,NaN,NaN,0,None,1
142181,3561384,Test,451664975_1722933822,100012776_37918976071_457913,confirm,2017-04-26 13:23:09,0 days 00:00:52,52.0,4,4.0,confirm,0,None,1
315155,7338123,Test,612065484_94198474375,100019538_17884295066_43909,start,2017-04-09 16:20:56,0 days 00:00:00,0.0,0,NaN,NaN,0,None,0
315154,7338123,Test,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:21:12,0 days 00:00:16,16.0,1,0.0,start,0,None,0
315153,7338123,Test,612065484_94198474375,100019538_17884295066_43909,step_2,2017-04-09 16:21:21,0 days 00:00:09,9.0,2,1.0,step_1,0,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127140,6627522,Test,730634087_44272418812,999988789_76411676596_272843,start,2017-04-21 23:49:11,0 days 00:00:00,0.0,0,NaN,NaN,0,None,0
127139,6627522,Test,730634087_44272418812,999988789_76411676596_272843,step_1,2017-04-21 23:49:22,0 days 00:00:11,11.0,1,0.0,start,0,None,0
127138,6627522,Test,730634087_44272418812,999988789_76411676596_272843,step_2,2017-04-21 23:50:16,0 days 00:00:54,54.0,2,1.0,step_1,0,None,0
127137,6627522,Test,730634087_44272418812,999988789_76411676596_272843,step_1,2017-04-21 23:51:00,0 days 00:00:44,44.0,1,2.0,step_2,1,step_2 → step_1,0


In [140]:
# We decide not to go with completion rate per user, rather per visit. We 'groupby' visit id, to sum the number of confirms per visit, 
# and we create a new dataframe

completed_users_per_group = df.groupby(['visit_id'])['confirm_step'].sum()
new_df = pd.DataFrame(completed_users_per_group).reset_index()
new_df

,visit_id,confirm_step
0,100012776_37918976071_457913,2
1,100019538_17884295066_43909,1
2,100022086_87870757897_149620,1
3,100030127_47967100085_936361,0
4,100037962_47432393712_705583,0
...,...,...
69200,999971096_28827267783_236076,1
69201,999976049_95772503197_182554,1
69202,999984454_18731538378_781808,1
69203,999985675_64610694964_443659,1


In [142]:
# We add groupby 'Variation' to make a distinction between test and control. 
# We create a new column called yes/no to make a distinction between visits that reached confirm status or not.

completed_users_per_group_var = df.groupby(['visit_id','Variation'])['confirm_step'].sum()
new_df_var = pd.DataFrame(completed_users_per_group_var).reset_index()
new_df_var['yes/no'] = new_df_var['confirm_step'].apply(lambda x:1 if x >= 1 else x)
new_df_var

,visit_id,Variation,confirm_step,yes/no
0,100012776_37918976071_457913,Test,2,1
1,100019538_17884295066_43909,Test,1,1
2,100022086_87870757897_149620,Test,1,1
3,100030127_47967100085_936361,Control,0,0
4,100037962_47432393712_705583,Control,0,0
...,...,...,...,...
69320,999971096_28827267783_236076,Test,1,1
69321,999976049_95772503197_182554,Test,1,1
69322,999984454_18731538378_781808,Test,1,1
69323,999985675_64610694964_443659,Control,1,1


In [144]:
new_df_var.to_excel('completion_rate.xlsx')

In [27]:
# we move forward to calculate completion rate for both variation groups separately

completed_rate_Control = new_df_var[new_df_var['Variation']== 'Control']
completed_rate_Control

,visit_id,Variation,confirm_step,yes/no
3,100030127_47967100085_936361,Control,0,0
4,100037962_47432393712_705583,Control,0,0
5,100057941_88477660212_944512,Control,0,0
6,10006594_66157970412_679648,Control,1,1
7,100071743_53464757454_616703,Control,0,0
...,...,...,...,...
69309,999859408_41720215615_938916,Control,1,1
69310,999890184_77121766521_746360,Control,0,0
69314,999936732_27995195904_324897,Control,0,0
69316,999951892_54774917180_163890,Control,0,0


In [31]:
cr_control = completed_rate_Control['yes/no'].sum() / completed_rate_Control['visit_id'].nunique()
cr_control

0.4984932740998478

In [33]:
completed_rate_Test = new_df_var[new_df_var['Variation']== 'Test']
completed_rate_Test

,visit_id,Variation,confirm_step,yes/no
0,100012776_37918976071_457913,Test,2,1
1,100019538_17884295066_43909,Test,1,1
2,100022086_87870757897_149620,Test,1,1
11,100173292_91322748906_143563,Test,1,1
13,100217156_67053490690_383412,Test,1,1
...,...,...,...,...
69319,999960019_60838685252_926860,Test,0,0
69320,999971096_28827267783_236076,Test,1,1
69321,999976049_95772503197_182554,Test,1,1
69322,999984454_18731538378_781808,Test,1,1


In [37]:
cr_test = completed_rate_Test['yes/no'].sum() / completed_rate_Test['visit_id'].nunique()
cr_test

0.5851734166307626

In [39]:
# Same process we did, but without the var column. We aim to calculate the completion rate for both grouops combined.
new_df['yes/no'] = new_df['confirm_step'].apply(lambda x:1 if x >= 1 else x)
new_df

,visit_id,confirm_step,yes/no
0,100012776_37918976071_457913,2,1
1,100019538_17884295066_43909,1,1
2,100022086_87870757897_149620,1,1
3,100030127_47967100085_936361,0,0
4,100037962_47432393712_705583,0,0
...,...,...,...
69200,999971096_28827267783_236076,1,1
69201,999976049_95772503197_182554,1,1
69202,999984454_18731538378_781808,1,1
69203,999985675_64610694964_443659,1,1


In [41]:
completion_rate  = new_df['yes/no'].sum()/new_df['visit_id'].nunique() 
completion_rate

0.5444693302507044

In [43]:
cr = completion_rate

In [45]:
# we have all the completion rates now, divided by variation group and in total.
cr, cr_test, cr_control

(0.5444693302507044, 0.5851734166307626, 0.4984932740998478)

# Average Time Per Step

In [23]:
# 1. Make sure the date_time column is in proper datetime format:

df['date_time'] = pd.to_datetime(df['date_time'])

In [25]:
# 2. Sort by visit_id and date_time to ensure events are in chronological order for each session:

df = df.sort_values(by=['visit_id', 'date_time'])
df

,client_id,Variation,visitor_id,visit_id,process_step,date_time
142182,3561384,Test,451664975_1722933822,100012776_37918976071_457913,confirm,2017-04-26 13:22:17
142181,3561384,Test,451664975_1722933822,100012776_37918976071_457913,confirm,2017-04-26 13:23:09
315155,7338123,Test,612065484_94198474375,100019538_17884295066_43909,start,2017-04-09 16:20:56
315154,7338123,Test,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:21:12
315153,7338123,Test,612065484_94198474375,100019538_17884295066_43909,step_2,2017-04-09 16:21:21
...,...,...,...,...,...,...
127140,6627522,Test,730634087_44272418812,999988789_76411676596_272843,start,2017-04-21 23:49:11
127139,6627522,Test,730634087_44272418812,999988789_76411676596_272843,step_1,2017-04-21 23:49:22
127138,6627522,Test,730634087_44272418812,999988789_76411676596_272843,step_2,2017-04-21 23:50:16
127137,6627522,Test,730634087_44272418812,999988789_76411676596_272843,step_1,2017-04-21 23:51:00


In [27]:
# 3. Calculate time difference to previous row (this will become the duration of the previous step)

df['time_diff'] = df.groupby('visit_id')['date_time'].diff()

In [29]:
# 4. Fill first row per session with 0 (since there's no time_diff before the first step)

df['time_diff'] = df['time_diff'].fillna(pd.Timedelta(seconds=0))

In [35]:
# 5. Convert to minutes for readability

df['time_diff_min'] = df['time_diff'].dt.total_seconds()

In [39]:
df.to_excel('avg_time_step.xlsx')

In [43]:
# 7. Group by process_step and sum durations

step_times = df.groupby('process_step')['time_diff_min'].sum().reset_index()

In [45]:
# Round
step_times['time_diff_min'] = step_times['time_diff_min'].round(2)

In [47]:
# average time per step
avg_step_times = df.groupby('process_step')['time_diff_min'].mean().reset_index()

In [49]:
step_times

,process_step,time_diff_min
0,confirm,5033052.0
1,start,6076698.0
2,step_1,2711989.0
3,step_2,2479868.0
4,step_3,4584488.0


In [51]:
avg_step_times

,process_step,time_diff_min
0,confirm,117.222191
1,start,60.074323
2,step_1,39.759405
3,step_2,43.758258
4,step_3,94.987734


In [134]:
# Convert decimal minutes to MM:SS format
def format_minutes_seconds(minutes):
    total_seconds = int(round(minutes * 60))
    mins, secs = divmod(total_seconds, 60)
    return f"{mins:02d}:{secs:02d}"

avg_step_times['time_mm_ss'] = avg_step_times['time_diff_min'].apply(format_minutes_seconds)

# Time Spent on Each Step

In [136]:
avg_step_times

,process_step,time_diff_min,time_mm_ss
0,confirm,1.953703,01:57
1,start,1.001239,01:00
2,step_1,0.662657,00:40
3,step_2,0.729304,00:44
4,step_3,1.583129,01:35


In [137]:
# 1. Group by process_step and variation
avg_step_by_variation = df.groupby(['process_step', 'Variation'])['time_diff_min'].mean().reset_index()

avg_step_by_variation['time_mm_ss'] = avg_step_by_variation['time_diff_min'].apply(format_minutes_seconds)

In [138]:
# Pivot to show Test and Control side-by-side
pivot_mmss = avg_step_by_variation.pivot(index='process_step', columns='Variation', values='time_mm_ss').reset_index()

# Optional: rename columns for clarity
pivot_mmss.columns.name = None
pivot_mmss = pivot_mmss.rename(columns={'Control': 'Control (MM:SS)', 'Test': 'Test (MM:SS)'})

In [139]:
pivot_mmss

,process_step,Control (MM:SS),Test (MM:SS)
0,confirm,02:04,01:52
1,start,00:57,01:03
2,step_1,00:43,00:37
3,step_2,00:39,00:48
4,step_3,01:33,01:36


In [140]:
# NOTE: If users visit the same step multiple times in one session, we are summing all those durations. 

# Error rate

In [53]:
# 1. Map steps to numeric order
step_order = ['start', 'step_1', 'step_2', 'step_3', 'confirm']
step_to_num = {step: i for i, step in enumerate(step_order)}
df['step_num'] = df['process_step'].map(step_to_num)

# 2. Sort by visit and date
df = df.sort_values(by=['visit_id', 'date_time'])

# 3. Get previous step number and name
df['prev_step_num'] = df.groupby('visit_id')['step_num'].shift(1)
df['prev_step_name'] = df.groupby('visit_id')['process_step'].shift(1)

# 4. Compare step numbers to detect backward movement (error)
df['error_flag'] = (df['step_num'] < df['prev_step_num']).astype(int)

# 5. Create a column to capture error transitions (e.g., "step_3 → step_2")
df['error_transition'] = df.apply(
    lambda row: f"{row['prev_step_name']} → {row['process_step']}" if row['error_flag'] == 1 else None,
    axis=1
)

In [68]:
df

,client_id,Variation,visitor_id,visit_id,process_step,date_time,time_diff,time_diff_min,step_num,prev_step_num,prev_step_name,error_flag,error_transition
142182,3561384,Test,451664975_1722933822,100012776_37918976071_457913,confirm,2017-04-26 13:22:17,0 days 00:00:00,0.0,4,NaN,NaN,0,None
142181,3561384,Test,451664975_1722933822,100012776_37918976071_457913,confirm,2017-04-26 13:23:09,0 days 00:00:52,52.0,4,4.0,confirm,0,None
315155,7338123,Test,612065484_94198474375,100019538_17884295066_43909,start,2017-04-09 16:20:56,0 days 00:00:00,0.0,0,NaN,NaN,0,None
315154,7338123,Test,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:21:12,0 days 00:00:16,16.0,1,0.0,start,0,None
315153,7338123,Test,612065484_94198474375,100019538_17884295066_43909,step_2,2017-04-09 16:21:21,0 days 00:00:09,9.0,2,1.0,step_1,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
127140,6627522,Test,730634087_44272418812,999988789_76411676596_272843,start,2017-04-21 23:49:11,0 days 00:00:00,0.0,0,NaN,NaN,0,None
127139,6627522,Test,730634087_44272418812,999988789_76411676596_272843,step_1,2017-04-21 23:49:22,0 days 00:00:11,11.0,1,0.0,start,0,None
127138,6627522,Test,730634087_44272418812,999988789_76411676596_272843,step_2,2017-04-21 23:50:16,0 days 00:00:54,54.0,2,1.0,step_1,0,None
127137,6627522,Test,730634087_44272418812,999988789_76411676596_272843,step_1,2017-04-21 23:51:00,0 days 00:00:44,44.0,1,2.0,step_2,1,step_2 → step_1


In [70]:
df_err_cri = df[['visit_id', 'Variation', 'process_step', 'error_flag']]

In [74]:
df_err_cri['confirm'] = df_err_cri['process_step'].apply(lambda x: 1 if x=='confirm' else 0)

/var/folders/y6/yy6xx0td03j17rtqbfny6l900000gn/T/ipykernel_92384/554791114.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_err_cri['confirm'] = df_err_cri['process_step'].apply(lambda x: 1 if x=='confirm' else 0)


In [118]:
df_error = df_err_cri.groupby(['visit_id', 'Variation'])[['error_flag', 'confirm']].sum().reset_index()
df_error

,visit_id,Variation,error_flag,confirm
0,100012776_37918976071_457913,Test,0,2
1,100019538_17884295066_43909,Test,2,1
2,100022086_87870757897_149620,Test,0,1
3,100030127_47967100085_936361,Control,0,0
4,100037962_47432393712_705583,Control,1,0
...,...,...,...,...
69320,999971096_28827267783_236076,Test,0,1
69321,999976049_95772503197_182554,Test,0,1
69322,999984454_18731538378_781808,Test,0,1
69323,999985675_64610694964_443659,Control,0,1


In [110]:
df_error_cri = df_error[df_error['error_flag'] > 0].shape[0]
df_error_cri

16645

In [114]:
tot_num_visits = df_error.shape[0]

In [94]:
visits_errors_no_confirm = df_error[(df_error['error_flag'] > 0) & (df_error['confirm'] == 0)].shape[0]

In [146]:
visits_errors_no_confirm

8818

In [98]:
err_criticality = visits_errors_no_confirm / tot_num_visits
err_criticality

0.12719798052650558

In [116]:
err_criticality = visits_errors_no_confirm / df_error_cri
err_criticality

0.5297686993091019

In [122]:
# Visits with errors within the test group

df_error_cri_test = df_error[(df_error['error_flag'] > 0) & (df_error['Variation'] == 'Test')].shape[0]
df_error_cri_test

10038

In [128]:
# number of visits with errors that didn't reach confirm for test

visits_errors_no_confirm_test = df_error[(df_error['error_flag'] > 0) & (df_error['confirm'] == 0) & (df_error['Variation'] == 'Test')].shape[0]
visits_errors_no_confirm_test

5750

In [124]:
# Visits with errors within the control group
df_error_cri_control = df_error[(df_error['error_flag'] > 0) & (df_error['Variation'] == 'Control')].shape[0]
df_error_cri_control

6607

In [130]:
# number of visits with errors that didn't reach confirm for control

visits_errors_no_confirm_control = df_error[(df_error['error_flag'] > 0) & (df_error['confirm'] == 0) & (df_error['Variation'] == 'Control')].shape[0]
visits_errors_no_confirm_control

3068

In [132]:
# Error criticality Test
visits_errors_no_confirm_test/df_error_cri_test

0.5728232715680415

In [134]:
# Error criticality Control
visits_errors_no_confirm_control/df_error_cri_control

0.4643559860753746

#### Error rate (% visits with at least one error)

In [59]:
# Whether each visit had at least one error
visit_errors = df.groupby('visit_id')['error_flag'].max().reset_index()
visit_errors.rename(columns={'error_flag': 'visit_has_error'}, inplace=True)

,client_id,Variation,visitor_id,visit_id,process_step,date_time,time_diff,time_diff_min,step_num,prev_step_num,prev_step_name,error_flag,error_transition
142182,3561384,Test,451664975_1722933822,100012776_37918976071_457913,confirm,2017-04-26 13:22:17,0 days 00:00:00,0.0,4,NaN,NaN,0,None
142181,3561384,Test,451664975_1722933822,100012776_37918976071_457913,confirm,2017-04-26 13:23:09,0 days 00:00:52,52.0,4,4.0,confirm,0,None
315155,7338123,Test,612065484_94198474375,100019538_17884295066_43909,start,2017-04-09 16:20:56,0 days 00:00:00,0.0,0,NaN,NaN,0,None
315154,7338123,Test,612065484_94198474375,100019538_17884295066_43909,step_1,2017-04-09 16:21:12,0 days 00:00:16,16.0,1,0.0,start,0,None
315153,7338123,Test,612065484_94198474375,100019538_17884295066_43909,step_2,2017-04-09 16:21:21,0 days 00:00:09,9.0,2,1.0,step_1,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
127140,6627522,Test,730634087_44272418812,999988789_76411676596_272843,start,2017-04-21 23:49:11,0 days 00:00:00,0.0,0,NaN,NaN,0,None
127139,6627522,Test,730634087_44272418812,999988789_76411676596_272843,step_1,2017-04-21 23:49:22,0 days 00:00:11,11.0,1,0.0,start,0,None
127138,6627522,Test,730634087_44272418812,999988789_76411676596_272843,step_2,2017-04-21 23:50:16,0 days 00:00:54,54.0,2,1.0,step_1,0,None
127137,6627522,Test,730634087_44272418812,999988789_76411676596_272843,step_1,2017-04-21 23:51:00,0 days 00:00:44,44.0,1,2.0,step_2,1,step_2 → step_1


In [65]:
df.error_flag.unique()

array([0, 1])

In [63]:
df.to_excel('visit_errors.xlsx')

In [61]:
visit_errors

,visit_id,visit_has_error
0,100012776_37918976071_457913,0
1,100019538_17884295066_43909,1
2,100022086_87870757897_149620,0
3,100030127_47967100085_936361,0
4,100037962_47432393712_705583,1
...,...,...
69200,999971096_28827267783_236076,0
69201,999976049_95772503197_182554,0
69202,999984454_18731538378_781808,0
69203,999985675_64610694964_443659,0


In [ ]:
# Total visits and error rate
total_visits = visit_errors.shape[0]
visits_with_errors = visit_errors['visit_has_error'].sum()
error_rate = visits_with_errors / total_visits

In [145]:
# % of visits with at least one error
error_rate

0.24014160826529876

#### Average Number of Errors (per visit that had errors)

In [147]:
# Count errors per visit
errors_per_visit = df.groupby('visit_id')['error_flag'].sum().reset_index()

# Merge with visit error info
errors_per_visit = errors_per_visit.merge(visit_errors, on='visit_id')

# Only visits that had at least one error
errors_with_errors = errors_per_visit[errors_per_visit['visit_has_error'] == 1]

# Average number of errors per error-visit
avg_errors_per_error_visit = errors_with_errors['error_flag'].mean()

In [148]:
avg_errors_per_error_visit

1.5668812804621217

#### Most common error transition

In [150]:
# Count frequency of each error transition
most_common_error = (
    df[df['error_flag'] == 1]
    .groupby('error_transition')
    .size()
    .reset_index(name='count')
    .sort_values(by='count', ascending=False)
)

In [151]:
most_common_error

,error_transition,count
4,step_1 → start,8903
9,step_3 → step_2,4646
6,step_2 → step_1,4532
7,step_3 → start,3304
5,step_2 → start,2418
8,step_3 → step_1,1041
0,confirm → start,928
1,confirm → step_1,144
3,confirm → step_3,123
2,confirm → step_2,1


# Error criticality

#### How likely is it that an error prevents a user from reaching the confirm step?

In [153]:
# Whether each visit had at least one error
visit_errors = df.groupby('visit_id')['error_flag'].max().reset_index()
visit_errors.rename(columns={'error_flag': 'visit_has_error'}, inplace=True)

# Whether each visit reached confirm
visit_reached_confirm = df[df['process_step'] == 'confirm'][['visit_id']].drop_duplicates()
visit_reached_confirm['reached_confirm'] = 1

# Merge both
visit_summary = visit_errors.merge(visit_reached_confirm, on='visit_id', how='left')
visit_summary['reached_confirm'] = visit_summary['reached_confirm'].fillna(0).astype(int)

In [188]:
#  Aggregate to get the combinations
error_vs_confirm = (
    visit_summary
    .groupby(['visit_has_error', 'reached_confirm'])
    .size()
    .reset_index(name='count')
)

In [194]:
# Calculate total visits
total_visits = error_vs_confirm['count'].sum()

# Compute percentage of total visits
error_vs_confirm['pct_of_total'] = (
    error_vs_confirm['count'] / total_visits * 100
).round(2)

In [200]:
error_vs_confirm

,visit_has_error,reached_confirm,count,pct_of_total
0,0,0,22721,32.83
1,0,1,29865,43.15
2,1,0,8804,12.72
3,1,1,7815,11.29
